In [1]:
import pandas as pd
import json

In [2]:
sentences = pd.read_feather('artefacts/pandas_frames/parlamint_sentences.feather')
metadata = pd.read_feather('artefacts/pandas_frames/parlamint_metadata.feather')

In [3]:
metadata

,index,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,Subcorpus,Speaker_role,Speaker_MP,Speaker_Minister,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
0,0,ParlaMint-ES-GA_2017-09-26-DSPG040.u1,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
1,1,ParlaMint-ES-GA_2017-09-26-DSPG040.u2,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
2,2,ParlaMint-ES-GA_2017-09-26-DSPG040.u3,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Regular,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Arias Rodríguez, Raquel",F,1966
3,3,ParlaMint-ES-GA_2017-09-26-DSPG040.u4,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
4,4,ParlaMint-ES-GA_2017-09-26-DSPG040.u5,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365545,213,ParlaMint-UA_2019-11-13-m0.u214,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Олексійович Гончаренко, Олексій",M,1980
1365546,214,ParlaMint-UA_2019-11-13-m0.u215,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983
1365547,215,ParlaMint-UA_2019-11-13-m0.u216,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Володимирівна Геращенко, Ірина",F,1971
1365548,216,ParlaMint-UA_2019-11-13-m0.u217,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983


In [4]:
sentences['lemmas'] = sentences.lemmas_tf.apply(
    lambda x: ' '.join([' '.join([f] * t) for f, t in json.loads(x).items()]))
sentences

,sentence_id,speech_id,language,text,lemmas_tf,lemmas
0,ParlaMint-ES-GA_2017-02-21-DSPG017.seg1.s1,ParlaMint-ES-GA_2017-02-21-DSPG017.u1,ES-GA,"Bos días, outra vez.","{""bo"": 1, ""d\u00eda"": 1, ""outro"": 1, ""vez"": 1}",bo día outro vez
1,ParlaMint-ES-GA_2017-02-21-DSPG017.seg2.s1,ParlaMint-ES-GA_2017-02-21-DSPG017.u1,ES-GA,Ábrese a sesión.,"{""o"": 1, ""sesi\u00f3n"": 1}",o sesión
2,ParlaMint-ES-GA_2017-02-21-DSPG017.seg3.s1,ParlaMint-ES-GA_2017-02-21-DSPG017.u1,ES-GA,A orde do día está publicada no Boletín Oficia...,"{""o"": 1, ""orde"": 1, ""d\u00eda"": 1, ""estar"": 1,...",o orde día estar publicar boletín oficial parl...
3,ParlaMint-ES-GA_2017-02-21-DSPG017.seg3.s2,ParlaMint-ES-GA_2017-02-21-DSPG017.u1,ES-GA,Non se xustificaron inasistencias.,"{""non"": 1, ""se"": 1, ""xustificar"": 1, ""inasiste...",non se xustificar inasistencias
4,ParlaMint-ES-GA_2017-02-21-DSPG017.seg4.s1,ParlaMint-ES-GA_2017-02-21-DSPG017.u1,ES-GA,Antes de dar comezo ao desenvolvemento da orde...,"{""antes"": 1, ""de"": 3, ""dar"": 1, ""comezo"": 1, ""...",antes de de de dar comezo desenvolvemento orde...
...,...,...,...,...,...,...
13107427,ParlaMint-UA_2019-04-23-m1.u256.p2.s8,ParlaMint-UA_2019-04-23-m1.u256,UA,"Я наголошую: ніяких відряджень, ніяких відпусток.","{""\u044f"": 1, ""\u043d\u0430\u0433\u043e\u043b\...",я наголошувати ніякий ніякий відрядження відпу...
13107428,ParlaMint-UA_2019-04-23-m1.u256.p2.s9,ParlaMint-UA_2019-04-23-m1.u256,UA,До мене приходило кілька депутатів з відпустка...,"{""\u0434\u043e"": 1, ""\u044f"": 1, ""\u043f\u0440...",до я приходити кілька депутат з відпустка відк...
13107429,ParlaMint-UA_2019-04-23-m1.u256.p2.s10,ParlaMint-UA_2019-04-23-m1.u256,UA,"Відкладіть, хай усі до одного будуть в залі, і...","{""\u0432\u0456\u0434\u043a\u043b\u0430\u0441\u...",відкласти хай увесь до один бути в зал і я впе...
13107430,ParlaMint-UA_2019-04-23-m1.u256.p3.s1,ParlaMint-UA_2019-04-23-m1.u256,UA,Зараз вечірнє засідання Верховної Ради України...,"{""\u0437\u0430\u0440\u0430\u0437"": 1, ""\u0432\...",зараз вечірній засідання верховний рада Україн...


In [5]:
sentences_gb = sentences[(sentences.language == 'GB') & (sentences.speech_id.str.contains('commons'))]
metadata_gb = metadata[metadata.ID.isin(sentences_gb.speech_id.unique())]

In [6]:
may_1_to_32_2020_metadata = metadata_gb[(metadata_gb.Date >= '2020-05-01') & (metadata_gb.Date <= '2020-05-31')]
may_1_to_32_2020_sentences = sentences_gb[sentences_gb.speech_id.isin(may_1_to_32_2020_metadata.ID)]
may_1_to_32_2020_metadata.Date.unique()

array(['2020-05-04', '2020-05-20', '2020-05-18', '2020-05-11',
       '2020-05-05', '2020-05-06', '2020-05-12', '2020-05-19',
       '2020-05-13'], dtype=object)

In [7]:
con_sentences = may_1_to_32_2020_sentences[may_1_to_32_2020_sentences.speech_id.isin(
    may_1_to_32_2020_metadata[may_1_to_32_2020_metadata.Speaker_party == 'CON'].ID.unique())]
lab_sentences = may_1_to_32_2020_sentences[may_1_to_32_2020_sentences.speech_id.isin(
    may_1_to_32_2020_metadata[may_1_to_32_2020_metadata.Speaker_party == 'LAB'].ID.unique())]
'con:', len(con_sentences), 'lab:', len(lab_sentences)

('con:', 10021, 'lab:', 4672)

In [25]:
# get lemma groups by speach_id
lemmas = [
    ('con', ' '.join(con_sentences.groupby('speech_id').lemmas.apply(lambda x: ' '.join(x)))),
    ('lab', ' '.join(lab_sentences.groupby('speech_id').lemmas.apply(lambda x: ' '.join(x)))),
]

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_df=5)

documents = [l for _, l in lemmas]
X = vectorizer.fit_transform(documents)

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

tfidf_df = tfidf_df.T
top_words = {}
for i in range(len(documents)):
    sorted_df = tfidf_df[i].sort_values(ascending=False)
    top_words[lemmas[i][0]] = sorted_df.index.tolist()
    top_words[f'{lemmas[i][0]}_scores'] = sorted_df.tolist()
pd.DataFrame(top_words)

,con,con_scores,lab,lab_scores
0,work,0.280025,government,0.361639
1,people,0.235918,work,0.187321
2,make,0.210132,people,0.184431
3,support,0.209453,minister,0.175764
4,right,0.206739,make,0.165651
...,...,...,...,...
8559,perplex,0.000000,plc,0.000000
8560,perpetrator,0.000000,doogan,0.000000
8561,pernicious,0.000000,doncaster,0.000000
8562,permanently,0.000000,don,0.000000
